# Christmas Song Recommender


### Load libraries

In [1]:
import pandas as pd
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random

### Import data

In [2]:
# Load liked songs from API extraction
# (most recent 100 user liked songs)
liked_songs = pd.read_csv('Project Data/liked_songs.csv')
(liked_songs.head())

,id,name,artists,album,release_date,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2NFadq6pUeiVEihLvUlOSr,"Rauw Alejandro: Bzrp Music Sessions, Vol. 56","Bizarrap, Rauw Alejandro","Rauw Alejandro: Bzrp Music Sessions, Vol. 56",2023-06-21,65,0.774,0.572,11,-6.000,...,0.003260,0.3150,0.489,128.010,audio_features,spotify:track:2NFadq6pUeiVEihLvUlOSr,https://api.spotify.com/v1/tracks/2NFadq6pUeiV...,https://api.spotify.com/v1/audio-analysis/2NFa...,191254,4
1,3A1BJKGMsa8JqO8M8zztyq,Me Gustas Tu,Manu Chao,Me Gustas Tu EP,2023-02-13,59,0.657,0.742,11,-6.822,...,0.493000,0.2220,0.909,175.943,audio_features,spotify:track:3A1BJKGMsa8JqO8M8zztyq,https://api.spotify.com/v1/tracks/3A1BJKGMsa8J...,https://api.spotify.com/v1/audio-analysis/3A1B...,239987,4
2,2lwwrWVKdf3LR9lbbhnr6R,Float On,Modest Mouse,Good News For People Who Love Bad News,2004-04-05,71,0.649,0.888,6,-4.807,...,0.000002,0.0888,0.553,100.975,audio_features,spotify:track:2lwwrWVKdf3LR9lbbhnr6R,https://api.spotify.com/v1/tracks/2lwwrWVKdf3L...,https://api.spotify.com/v1/audio-analysis/2lww...,208467,4
3,77RNVzAVwqWiZrZuWqMV2i,Nancy From Now On,Father John Misty,Fear Fun,2012-05-01,61,0.535,0.646,0,-9.168,...,0.040600,0.0908,0.501,107.078,audio_features,spotify:track:77RNVzAVwqWiZrZuWqMV2i,https://api.spotify.com/v1/tracks/77RNVzAVwqWi...,https://api.spotify.com/v1/audio-analysis/77RN...,234160,4
4,6uW4QhvYTl6q24OBbWTt9s,Convite,Lamparina,Manda Dizer,2018-08-06,44,0.470,0.621,11,-7.369,...,0.000000,0.1270,0.618,117.952,audio_features,spotify:track:6uW4QhvYTl6q24OBbWTt9s,https://api.spotify.com/v1/tracks/6uW4QhvYTl6q...,https://api.spotify.com/v1/audio-analysis/6uW4...,213384,4


### User vector top 100 songs

In [3]:
# Take average of each of the audio features to get user vector
popularity_avg = liked_songs['popularity'].mean()
danceability_avg = liked_songs['danceability'].mean()
energy_avg = liked_songs['energy'].mean()
key_avg = liked_songs['key'].mean()
loudness_avg = liked_songs['loudness'].mean()
mode_avg = liked_songs['mode'].mean()
speechiness_avg = liked_songs['speechiness'].mean()
acousticness_avg = liked_songs['acousticness'].mean()
instrumentalness_avg = liked_songs['instrumentalness'].mean()
liveness_avg = liked_songs['liveness'].mean()
valence_avg = liked_songs['valence'].mean()
tempo_avg = liked_songs['tempo'].mean()
duration_avg = liked_songs['duration_ms'].mean()
time_avg = liked_songs['time_signature'].mean()

In [4]:
user_vector = [popularity_avg,danceability_avg,energy_avg,key_avg,loudness_avg,mode_avg,speechiness_avg,acousticness_avg,
                instrumentalness_avg,liveness_avg,valence_avg,tempo_avg,duration_avg,time_avg]
user_vector

[58.63,
 0.59393,
 0.58185,
 5.21,
 -8.726320000000001,
 0.71,
 0.05485800000000001,
 0.36146888,
 0.09866979729999999,
 0.150994,
 0.50223,
 119.84297999999997,
 216932.31,
 3.9]

### Top 10 liked songs
We'll use both the user's most recent 100 and most recent 10 liked songs to generate recommendations to see if there is a difference.

In [5]:
liked_songs_10 = liked_songs.head(10)
liked_songs_10

,id,name,artists,album,release_date,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2NFadq6pUeiVEihLvUlOSr,"Rauw Alejandro: Bzrp Music Sessions, Vol. 56","Bizarrap, Rauw Alejandro","Rauw Alejandro: Bzrp Music Sessions, Vol. 56",2023-06-21,65,0.774,0.572,11,-6.000,...,0.003260,0.3150,0.489,128.010,audio_features,spotify:track:2NFadq6pUeiVEihLvUlOSr,https://api.spotify.com/v1/tracks/2NFadq6pUeiV...,https://api.spotify.com/v1/audio-analysis/2NFa...,191254,4
1,3A1BJKGMsa8JqO8M8zztyq,Me Gustas Tu,Manu Chao,Me Gustas Tu EP,2023-02-13,59,0.657,0.742,11,-6.822,...,0.493000,0.2220,0.909,175.943,audio_features,spotify:track:3A1BJKGMsa8JqO8M8zztyq,https://api.spotify.com/v1/tracks/3A1BJKGMsa8J...,https://api.spotify.com/v1/audio-analysis/3A1B...,239987,4
2,2lwwrWVKdf3LR9lbbhnr6R,Float On,Modest Mouse,Good News For People Who Love Bad News,2004-04-05,71,0.649,0.888,6,-4.807,...,0.000002,0.0888,0.553,100.975,audio_features,spotify:track:2lwwrWVKdf3LR9lbbhnr6R,https://api.spotify.com/v1/tracks/2lwwrWVKdf3L...,https://api.spotify.com/v1/audio-analysis/2lww...,208467,4
3,77RNVzAVwqWiZrZuWqMV2i,Nancy From Now On,Father John Misty,Fear Fun,2012-05-01,61,0.535,0.646,0,-9.168,...,0.040600,0.0908,0.501,107.078,audio_features,spotify:track:77RNVzAVwqWiZrZuWqMV2i,https://api.spotify.com/v1/tracks/77RNVzAVwqWi...,https://api.spotify.com/v1/audio-analysis/77RN...,234160,4
4,6uW4QhvYTl6q24OBbWTt9s,Convite,Lamparina,Manda Dizer,2018-08-06,44,0.470,0.621,11,-7.369,...,0.000000,0.1270,0.618,117.952,audio_features,spotify:track:6uW4QhvYTl6q24OBbWTt9s,https://api.spotify.com/v1/tracks/6uW4QhvYTl6q...,https://api.spotify.com/v1/audio-analysis/6uW4...,213384,4
5,0nrlsriZeFXhuFno9bVLNg,secrecy,Ethan Regan,secrecy,2023-08-23,49,0.642,0.338,1,-12.145,...,0.000239,0.1500,0.189,140.098,audio_features,spotify:track:0nrlsriZeFXhuFno9bVLNg,https://api.spotify.com/v1/tracks/0nrlsriZeFXh...,https://api.spotify.com/v1/audio-analysis/0nrl...,210000,4
6,5vbnQr1tWSSREmcIONIUm9,Backwards Directions,Abby Sage,Backwards Directions,2022-09-07,57,0.852,0.376,10,-10.434,...,0.452000,0.0956,0.484,114.999,audio_features,spotify:track:5vbnQr1tWSSREmcIONIUm9,https://api.spotify.com/v1/tracks/5vbnQr1tWSSR...,https://api.spotify.com/v1/audio-analysis/5vbn...,187072,4
7,2FwDApgXk91kXvqy2oB7dz,Simulation Swarm,Big Thief,Dragon New Warm Mountain I Believe In You,2022-02-11,66,0.703,0.578,1,-9.103,...,0.002400,0.1020,0.670,105.204,audio_features,spotify:track:2FwDApgXk91kXvqy2oB7dz,https://api.spotify.com/v1/tracks/2FwDApgXk91k...,https://api.spotify.com/v1/audio-analysis/2FwD...,252734,4
8,04mAOoQNsXmDJlyupJwmkO,Waiting Room,Phoebe Bridgers,"Lost Ark Studio Compilation, Vol. 8",2016-08-12,75,0.531,0.289,2,-10.065,...,0.000358,0.1340,0.120,129.753,audio_features,spotify:track:04mAOoQNsXmDJlyupJwmkO,https://api.spotify.com/v1/tracks/04mAOoQNsXmD...,https://api.spotify.com/v1/audio-analysis/04mA...,393962,4
9,7CoMBpPTwQi2wPT0U0Nr9b,State Lines,Novo Amor,Birthplace,2018-10-19,73,0.509,0.310,9,-10.623,...,0.229000,0.0963,0.151,137.474,audio_features,spotify:track:7CoMBpPTwQi2wPT0U0Nr9b,https://api.spotify.com/v1/tracks/7CoMBpPTwQi2...,https://api.spotify.com/v1/audio-analysis/7CoM...,208653,3


In [6]:
# Take average of each of the audio features to get user vector
popularity_avg10 = liked_songs_10['popularity'].mean()
danceability_avg10 = liked_songs_10['danceability'].mean()
energy_avg10 = liked_songs_10['energy'].mean()
key_avg10 = liked_songs_10['key'].mean()
loudness_avg10 = liked_songs_10['loudness'].mean()
mode_avg10 = liked_songs_10['mode'].mean()
speechiness_avg10 = liked_songs_10['speechiness'].mean()
acousticness_avg10 = liked_songs_10['acousticness'].mean()
instrumentalness_avg10 = liked_songs_10['instrumentalness'].mean()
liveness_avg10 = liked_songs_10['liveness'].mean()
valence_avg10 = liked_songs_10['valence'].mean()
tempo_avg10 = liked_songs_10['tempo'].mean()
duration_avg10 = liked_songs_10['duration_ms'].mean()
time_avg10 = liked_songs_10['time_signature'].mean()

In [7]:
user_vector_10 = [popularity_avg,danceability_avg,energy_avg,key_avg,loudness_avg,mode_avg,speechiness_avg,acousticness_avg,
                instrumentalness_avg,liveness_avg,valence_avg,tempo_avg,duration_avg,time_avg]
user_vector_10

[58.63,
 0.59393,
 0.58185,
 5.21,
 -8.726320000000001,
 0.71,
 0.05485800000000001,
 0.36146888,
 0.09866979729999999,
 0.150994,
 0.50223,
 119.84297999999997,
 216932.31,
 3.9]

### Load Christmas Data

In [8]:
# Load Christmas playlist data from Spotify API extraction
xmas_data = pd.read_csv('Project Data/xmas_data.csv')
(xmas_data.head())

,id,name,artists,album,release_date,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,2EjXfH91m7f8HiJN1yQg97,Rockin' Around The Christmas Tree,Brenda Lee,Merry Christmas From Brenda Lee,1964-10-19,83,0.598,0.470,8,-8.744,...,0.000000,0.5050,0.879,67.086,audio_features,spotify:track:2EjXfH91m7f8HiJN1yQg97,https://api.spotify.com/v1/tracks/2EjXfH91m7f8...,https://api.spotify.com/v1/audio-analysis/2EjX...,126267,4
1,7vQbuQcyTflfCIOu3Uzzya,Jingle Bell Rock,Bobby Helms,Jingle Bell Rock/Captain Santa Claus (And His ...,1957-12-02,80,0.754,0.424,2,-8.463,...,0.000000,0.0652,0.806,119.705,audio_features,spotify:track:7vQbuQcyTflfCIOu3Uzzya,https://api.spotify.com/v1/tracks/7vQbuQcyTflf...,https://api.spotify.com/v1/audio-analysis/7vQb...,130973,4
2,5hslUAKq9I9CG2bAulFkHN,It's the Most Wonderful Time of the Year,Andy Williams,The Andy Williams Christmas Album,1963-11-24,80,0.240,0.598,7,-8.435,...,0.000000,0.1170,0.776,201.629,audio_features,spotify:track:5hslUAKq9I9CG2bAulFkHN,https://api.spotify.com/v1/tracks/5hslUAKq9I9C...,https://api.spotify.com/v1/audio-analysis/5hsl...,151933,3
3,46pF1zFimM582ss1PrMy68,Christmas (Baby Please Come Home),Darlene Love,A Christmas Gift For You From Phil Spector,1963,66,0.325,0.759,8,-7.146,...,0.000362,0.0773,0.360,126.627,audio_features,spotify:track:46pF1zFimM582ss1PrMy68,https://api.spotify.com/v1/tracks/46pF1zFimM58...,https://api.spotify.com/v1/audio-analysis/46pF...,166453,4
4,4PS1e8f2LvuTFgUs1Cn3ON,The Christmas Song (Merry Christmas To You),Nat King Cole,The Christmas Song (Expanded Edition),1962,75,0.320,0.210,6,-15.231,...,0.000080,0.1380,0.208,73.196,audio_features,spotify:track:4PS1e8f2LvuTFgUs1Cn3ON,https://api.spotify.com/v1/tracks/4PS1e8f2LvuT...,https://api.spotify.com/v1/audio-analysis/4PS1...,192160,5


In [9]:
xmas_matrix = xmas_data.drop(['artists', 'name','album','release_date','type','uri','track_href','analysis_url'], axis=1)
# Number of rows
print(xmas_matrix.shape)
xmas_matrix.head()

(1960, 15)


,id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,2EjXfH91m7f8HiJN1yQg97,83,0.598,0.470,8,-8.744,1,0.0496,0.617,0.000000,0.5050,0.879,67.086,126267,4
1,7vQbuQcyTflfCIOu3Uzzya,80,0.754,0.424,2,-8.463,1,0.0363,0.643,0.000000,0.0652,0.806,119.705,130973,4
2,5hslUAKq9I9CG2bAulFkHN,80,0.240,0.598,7,-8.435,1,0.0369,0.766,0.000000,0.1170,0.776,201.629,151933,3
3,46pF1zFimM582ss1PrMy68,66,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,166453,4
4,4PS1e8f2LvuTFgUs1Cn3ON,75,0.320,0.210,6,-15.231,1,0.0345,0.920,0.000080,0.1380,0.208,73.196,192160,5


In [10]:
# Take the id variable out of the data frame so we have a matrix of all audio features
xmas_matrix.set_index('id', inplace=True)
xmas_matrix.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
id,,,,,,,,,,,,,,
2EjXfH91m7f8HiJN1yQg97,83,0.598,0.470,8,-8.744,1,0.0496,0.617,0.000000,0.5050,0.879,67.086,126267,4
7vQbuQcyTflfCIOu3Uzzya,80,0.754,0.424,2,-8.463,1,0.0363,0.643,0.000000,0.0652,0.806,119.705,130973,4
5hslUAKq9I9CG2bAulFkHN,80,0.240,0.598,7,-8.435,1,0.0369,0.766,0.000000,0.1170,0.776,201.629,151933,3
46pF1zFimM582ss1PrMy68,66,0.325,0.759,8,-7.146,1,0.0530,0.395,0.000362,0.0773,0.360,126.627,166453,4
4PS1e8f2LvuTFgUs1Cn3ON,75,0.320,0.210,6,-15.231,1,0.0345,0.920,0.000080,0.1380,0.208,73.196,192160,5


### Recommendations based on liked songs data:
Using the liked songs vector (average audio features of the user's liked songs). We will use cosine similarity to recommend similar Christmas songs to the user's preferences.

In [11]:
vector = np.array(user_vector)

In [12]:
vector_10 = np.array(user_vector_10)

In [13]:
import os
default_n_threads = 1
os.environ['OPENBLAS_NUM_THREADS'] = f"{default_n_threads}"
os.environ['MKL_NUM_THREADS'] = f"{default_n_threads}"
os.environ['OMP_NUM_THREADS'] = f"{default_n_threads}"
import sys


#### Get recommendations from top 100

In [22]:
# Reshape liked_features into a 2D array (to match the shape of the playlist data)
liked_features = vector.reshape(1, -1)

# Calculate cosine similarity between the liked_features and each song in the playlist
similarities = cosine_similarity(liked_features, xmas_matrix)

# Get the indices of the most similar songs (top N recommendations)
N = 6  # Number of songs to recommend
top_indices = similarities[0].argsort()[-N:][::-1]  # Sort indices by similarity, descending order

# Get the top N recommended songs
recommended_songs = xmas_matrix.iloc[top_indices]

print("Recommended Songs:")
print(recommended_songs)

Recommended Songs:
                        popularity  danceability  energy  key  loudness  mode  \
id                                                                              
0UOG0zUn7t8m8QcxfzR7AH          57         0.531   0.727    2    -9.268     1   
4fzyvSu73BhGvi96p2zwjL          68         0.315   0.418    7    -9.146     1   
0jXjTHqZVXhQSGUAvUbCvU          61         0.310   0.302    3   -14.485     1   
04DYwFIKeq2Bkn9aqSI9PC          53         0.270   0.254    4   -12.639     1   
2xXCYeiWBTy1h829foY6i1          42         0.614   0.782    1    -4.925     1   
7us9KL6g1bLBAoUSVu7Qzb          43         0.578   0.608    6    -6.961     1   

                        speechiness  acousticness  instrumentalness  liveness  \
id                                                                              
0UOG0zUn7t8m8QcxfzR7AH       0.0340         0.208          0.934000    0.0912   
4fzyvSu73BhGvi96p2zwjL       0.0299         0.131          0.000679    0.0836   
0jXjTHqZ

#### Get recommendations for top 10 vector

In [24]:
# Reshape liked_features into a 2D array (to match the shape of the playlist data)
liked_features = vector_10.reshape(1, -1)

# Calculate cosine similarity between the liked_features and each song in the playlist
similarities = cosine_similarity(liked_features, xmas_matrix)

# Get the indices of the most similar songs (top N recommendations)
N = 6  # Number of songs to recommend
top_indices = similarities[0].argsort()[-N:][::-1]  # Sort indices by similarity, descending order

# Get the top N recommended songs
recommended_songs = xmas_matrix.iloc[top_indices]

print("Recommended Songs:")
print(recommended_songs)

Recommended Songs:
                        popularity  danceability  energy  key  loudness  mode  \
id                                                                              
0UOG0zUn7t8m8QcxfzR7AH          57         0.531   0.727    2    -9.268     1   
4fzyvSu73BhGvi96p2zwjL          68         0.315   0.418    7    -9.146     1   
0jXjTHqZVXhQSGUAvUbCvU          61         0.310   0.302    3   -14.485     1   
04DYwFIKeq2Bkn9aqSI9PC          53         0.270   0.254    4   -12.639     1   
2xXCYeiWBTy1h829foY6i1          42         0.614   0.782    1    -4.925     1   
7us9KL6g1bLBAoUSVu7Qzb          43         0.578   0.608    6    -6.961     1   

                        speechiness  acousticness  instrumentalness  liveness  \
id                                                                              
0UOG0zUn7t8m8QcxfzR7AH       0.0340         0.208          0.934000    0.0912   
4fzyvSu73BhGvi96p2zwjL       0.0299         0.131          0.000679    0.0836   
0jXjTHqZ

#### Print Recommendations

In [25]:
rec_ids = recommended_songs.index.tolist()
xmas_data[xmas_data['id'].isin(rec_ids)][['name','artists']]

,name,artists
55,Adeste Fideles,Bing Crosby
97,When a Child Is Born,Johnny Mathis
132,Christmas Lights,Coldplay
289,Lit This Year,Florida Georgia Line
290,Have Yourself A Merry Little Christmas,Kacey Musgraves
396,"Merry Christmas, Please Don't Call",Bleachers


### Recommend songs based on one input song by user:

User gives a song ID from within the Christmas Playlist and system returns 5 other Christmas song recommendations.

In [18]:
# KNN model using cosine similarity
model_knn = NearestNeighbors(metric = "cosine", algorithm = "brute")
model_knn.fit(xmas_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [19]:
# CSR Matrix to save memory - compress matrix
data_matrix = csr_matrix(xmas_matrix.values)

In [20]:
id='0bYg9bo50gSsH3LtXe2SQn' # All I Want for Christmas is You 
distance, indice = model_knn.kneighbors([xmas_matrix.loc[id].values], n_neighbors=6)

print(distance)
print(indice)

[[1.11022302e-16 4.20339763e-10 5.26206745e-10 7.06713688e-10
  8.04151301e-10 1.15031629e-09]]
[[ 68 185 119 149 207 158]]


/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [21]:
# prints recommendations
rec_list = xmas_matrix.iloc[indice[0]].index.values.tolist()
xmas_data[xmas_data['id'].isin(rec_list)][['name','artists']]

,name,artists
68,All I Want for Christmas Is You,Mariah Carey
119,Someday At Christmas,Stevie Wonder
149,You Make It Feel Like Christmas (with Blake Sh...,"Gwen Stefani, Blake Shelton"
158,"You’re A Mean One, Mr. Grinch","Lindsey Stirling, Sabrina Carpenter"
185,buy me presents,Sabrina Carpenter
207,Blue Christmas,Megan Moroney
